In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
import plotly.express as px

**1. Carga y Exploración de Datos**

In [2]:
# Usamos la biblioteca google para poder usar archivos en nuestro drive.
from google.colab import drive
# Este comando conecta colab con drive.
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/Skillnest/ML/CORES/vehicles.parquet"
df = pd.read_parquet(path)

**2. LIMPIEZA Y PREPROCESAMIENTO**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [5]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,None,None,None,None,...,None,None,None,None,None,NaN,az,NaN,NaN,None
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,None,None,None,None,...,None,None,None,None,None,NaN,ar,NaN,NaN,None
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,None,None,None,None,...,None,None,None,None,None,NaN,fl,NaN,NaN,None
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,None,None,None,None,...,None,None,None,None,None,NaN,ma,NaN,NaN,None
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,None,None,None,None,...,None,None,None,None,None,NaN,nc,NaN,NaN,None


In [6]:
# Ver cuántos ids únicos hay
print("IDs únicos:", df["id"].nunique())
print("Total de filas:", len(df))

IDs únicos: 426880
Total de filas: 426880


No hay duplicados por id, mas adelante borraria esta columna

In [7]:
# Porcentaje de nulos por columna
porcentaje_nulos = df.isna().mean().sort_values(ascending=False) * 100

# Mostrar en forma de tabla
porcentaje_nulos = porcentaje_nulos.round(2).reset_index()
porcentaje_nulos.columns = ["Columna", "Porcentaje de Nulos"]
porcentaje_nulos

,Columna,Porcentaje de Nulos
0,county,100.00
1,size,71.77
2,cylinders,41.62
3,condition,40.79
4,VIN,37.73
5,drive,30.59
6,paint_color,30.50
7,type,21.75
8,manufacturer,4.13
9,title_status,1.93


In [8]:
df["url"]

,url
0,https://prescott.craigslist.org/cto/d/prescott...
1,https://fayar.craigslist.org/ctd/d/bentonville...
2,https://keys.craigslist.org/cto/d/summerland-k...
3,https://worcester.craigslist.org/cto/d/west-br...
4,https://greensboro.craigslist.org/cto/d/trinit...
...,...
426875,https://wyoming.craigslist.org/ctd/d/atlanta-2...
426876,https://wyoming.craigslist.org/ctd/d/atlanta-2...
426877,https://wyoming.craigslist.org/ctd/d/atlanta-2...
426878,https://wyoming.craigslist.org/ctd/d/atlanta-2...


In [9]:
df["region"]

,region
0,prescott
1,fayetteville
2,florida keys
3,worcester / central MA
4,greensboro
...,...
426875,wyoming
426876,wyoming
426877,wyoming
426878,wyoming


In [10]:
df["region_url"]

,region_url
0,https://prescott.craigslist.org
1,https://fayar.craigslist.org
2,https://keys.craigslist.org
3,https://worcester.craigslist.org
4,https://greensboro.craigslist.org
...,...
426875,https://wyoming.craigslist.org
426876,https://wyoming.craigslist.org
426877,https://wyoming.craigslist.org
426878,https://wyoming.craigslist.org


In [11]:
df["VIN"]

,VIN
0,None
1,None
2,None
3,None
4,None
...,...
426875,1N4AA6AV6KC367801
426876,7JR102FKXLG042696
426877,1GYFZFR46LF088296
426878,58ABK1GG4JU103853


In [12]:
df["size"]

,size
0,None
1,None
2,None
3,None
4,None
...,...
426875,None
426876,None
426877,None
426878,None


In [13]:
df["type"]

,type
0,None
1,None
2,None
3,None
4,None
...,...
426875,sedan
426876,sedan
426877,hatchback
426878,sedan


In [14]:
df["image_url"]

,image_url
0,None
1,None
2,None
3,None
4,None
...,...
426875,https://images.craigslist.org/00o0o_iiraFnHg8q...
426876,https://images.craigslist.org/00x0x_15sbgnxCIS...
426877,https://images.craigslist.org/00L0L_farM7bxnxR...
426878,https://images.craigslist.org/00z0z_bKnIVGLkDT...


In [15]:
df["description"]

,description
0,None
1,None
2,None
3,None
4,None
...,...
426875,Carvana is the safer way to buy a car During t...
426876,Carvana is the safer way to buy a car During t...
426877,Carvana is the safer way to buy a car During t...
426878,Carvana is the safer way to buy a car During t...


In [16]:
df["lat"]

,lat
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,33.786500
426876,33.786500
426877,33.779214
426878,33.786500


In [17]:
df["long"]

,long
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,-84.445400
426876,-84.445400
426877,-84.411811
426878,-84.445400


In [18]:
df["posting_date"]

,posting_date
0,None
1,None
2,None
3,None
4,None
...,...
426875,2021-04-04T03:21:31-0600
426876,2021-04-04T03:21:29-0600
426877,2021-04-04T03:21:17-0600
426878,2021-04-04T03:21:11-0600


Analizando las varibles anteriores de tipo texto y algunas númericas, además del porcentaje de valores nulos que poseen, procedo a eliminar las variables:

id, url, region_url, VIN, image_url y description, porque las considero pocos utiles en el analisis posterior, algunas son textos y otras son tipo codigos.

Tambien eliminare county y size, ya que poseen un gran porcentaje de valores nulos respecto al total, 100% y 71.77% respectivamente.

Lat y long representan la ubicacion, ademas a ambas les faltan los mismos valores, no las considero esenciales para el analisis porque son coordenadas exactas y para la ubicacion cuento con region y state.

De igual forma eliminare posting_date ya que contiene fecha y hora, y no es relevante en el analisis posterior.

In [19]:
# Eliminacion de columnas irrelevantes para el analisis, y con gran porcentaje de valores nulos.
columnas_a_eliminar = ["id", "url", "region_url", "VIN", "image_url", "description", "county", "size","lat","long","posting_date"]

df.drop(columns=columnas_a_eliminar, inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        426880 non-null  object 
 1   price         426880 non-null  int64  
 2   year          425675 non-null  float64
 3   manufacturer  409234 non-null  object 
 4   model         421603 non-null  object 
 5   condition     252776 non-null  object 
 6   cylinders     249202 non-null  object 
 7   fuel          423867 non-null  object 
 8   odometer      422480 non-null  float64
 9   title_status  418638 non-null  object 
 10  transmission  424324 non-null  object 
 11  drive         296313 non-null  object 
 12  type          334022 non-null  object 
 13  paint_color   296677 non-null  object 
 14  state         426880 non-null  object 
dtypes: float64(2), int64(1), object(12)
memory usage: 48.9+ MB


In [21]:
# Analizar estadisticas basicas
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
price,426880.0,75199.03,12182282.17,0.0,5900.0,13950.0,26485.75,3.736929e+09
year,425675.0,2011.24,9.45,1900.0,2008.0,2013.0,2017.00,2.022000e+03
odometer,422480.0,98043.33,213881.50,0.0,37704.0,85548.0,133542.50,1.000000e+07


Analizando las estadisticas basicas:

En price el maximo es muy elevado, mas adelante lo analizare, posible outlier. En year parece estar todo bien, año 1900 podrian ser autos clasicos, ademas los valores faltantes que hay los reemplazare por la mediana que es 2013 lo cual es mas representativa que la media. En odometer hay valores extremos igual, posibles outliers, mas adelante los analizo.

In [22]:
# Ver los 10 valores extremos en price
df[["price"]].sort_values(by="price", ascending=False).head(10)

,price
356716,3736928711
318592,3736928711
91576,3024942282
257840,3024942282
37410,3009548743
184704,1410065407
153082,1234567890
37409,1111111111
29386,1111111111
280,987654321


In [23]:
# Ver los 10 valores extremos en odometer
df[["odometer"]].sort_values(by="odometer", ascending=False).head(10)

,odometer
320410,10000000.0
110623,10000000.0
103798,10000000.0
108102,10000000.0
105059,10000000.0
346469,10000000.0
262338,10000000.0
9218,10000000.0
144436,10000000.0
413267,10000000.0


Luego de analizar que existen valores muy altos para price y odometer, decido eliminar esos valores atipicos segun el percentil 99

Los valores más allá del percentil 99 suelen ser outliers o errores, los cuales pueden distorsionar el análisis o el entrenamiento del modelo, lo que se hara es, filtrar solo el 1% más alto, entonces se eliminan pocos datos, por ende no se pierde mucha información.

In [24]:
# Eliminar valores otuliers para price y odometer

# Calcular percentil 99 para price y odometer
umbral_precio = df["price"].quantile(0.99)
umbral_odometer = df["odometer"].quantile(0.99)

# Filtrar para eliminar outliers en price y odometer
df_limpio = df[(df["price"] <= umbral_precio) & (df["odometer"] <= umbral_odometer)]

In [25]:
# Reemplazar por moda lo valores nulos en year y odometer
df_limpio.loc[:, "year"] = df_limpio["year"].fillna(df_limpio["year"].median())
df_limpio.loc[:, "odometer"] = df_limpio["odometer"].fillna(df_limpio["odometer"].median())

In [26]:
# Imputar columnas cateogircas con la moda
columnas_categoricas_con_nulos = ["manufacturer", "model", "condition", "cylinders", "fuel","title_status", "transmission", "drive", "type", "paint_color"]

for col in columnas_categoricas_con_nulos:
    moda = df_limpio[col].mode()[0]
    df_limpio.loc[:, col] = df_limpio[col].fillna(moda)

In [27]:
# Verificacion final nulos
df_limpio.isna().sum()

,0
region,0
price,0
year,0
manufacturer,0
model,0
condition,0
cylinders,0
fuel,0
odometer,0
title_status,0


In [28]:
# Verificando
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 414169 entries, 27 to 426879
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        414169 non-null  object 
 1   price         414169 non-null  int64  
 2   year          414169 non-null  float64
 3   manufacturer  414169 non-null  object 
 4   model         414169 non-null  object 
 5   condition     414169 non-null  object 
 6   cylinders     414169 non-null  object 
 7   fuel          414169 non-null  object 
 8   odometer      414169 non-null  float64
 9   title_status  414169 non-null  object 
 10  transmission  414169 non-null  object 
 11  drive         414169 non-null  object 
 12  type          414169 non-null  object 
 13  paint_color   414169 non-null  object 
 14  state         414169 non-null  object 
dtypes: float64(2), int64(1), object(12)
memory usage: 50.6+ MB


**3. EXPLORACION DE DATOS**

In [29]:
df_limpio.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
price,414169.0,16905.55,13774.71,0.0,5977.0,13950.0,25990.0,66995.0
year,414169.0,2011.30,9.29,1900.0,2008.0,2013.0,2017.0,2022.0
odometer,414169.0,90294.55,60467.18,0.0,38138.0,85411.0,132217.0,280000.0


In [30]:
# Pie chart para distribución de fuel

fig = px.pie(
    df_limpio,
    names="fuel",
    title="Distribución de vehículos por tipo de combustible",
    hole=0.3  # gráfico tipo donut
)
fig.update_traces(textinfo="percent+label")
fig.update_layout(title_font_size=18)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

El gráfico muestra la proporción de vehículos según el tipo de combustible utilizado. Se observa que:

Existe un tipo predominante de combustible que es tipo gas con un 84.7%, lo que sugiere que la mayoría de los vehículos ofertados comparten esta característica.

Los combustibles como eléctrico, híbrido o diésel, tienen una participación minoritaria.

In [31]:
# Histograma para variable numérica price

fig = px.histogram(
    df_limpio,
    x="price",
    nbins=30,
    title="Distribución de precios de vehículos usados"
)
fig.update_layout(bargap=0.1, title_font_size=18)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

El histograma muestra cómo se distribuyen los precios de los vehículos en el dataset, la mayoría de los vehículos se concentran en el rango de precios bajos a medios, entre aproximadamente 0 y 30k.

A medida que el precio aumenta, la frecuencia disminuye drásticamente, lo que indica que los vehículos más caros son menos comunes.

In [32]:
# Histograma agrupado para precios de vehículos por combustible y tipo de transmisión

fig = px.histogram(
    df_limpio,
    x="fuel",
    y="price",
    color="transmission",
    barmode="group",
    title="Distribución de precios de vehículos usados según tipo de combustible y transmisión"
)
fig.update_layout(title_font_size=18)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

Este gráfico muestra cómo varía el precio promedio de los vehículos según el tipo de combustible (fuel) y la transmisión (transmission):

En general, los vehículos con transmisión automática tienden a tener precios más altos en casi todos los tipos de combustible.

El tipo de combustible también influye: por ejemplo, algunos combustibles como eléctrico o híbrido presentan menos datos, posiblemente por su tecnología y menor disponibilidad.

Se pueden observar diferencias claras entre subgrupos: por ejemplo, dentro del combustible "gasolina", la transmisión automática supera consistentemente en precio a la manual.

**4. MODELADO Y EVALUACION**

Entrenar y evaluar Linear Regression y Random Forest

In [33]:
df_limpio["condition"].unique()

array(['good', 'excellent', 'fair', 'like new', 'new', 'salvage'],
      dtype=object)

In [34]:
df_limpio["cylinders"].unique()

array(['8 cylinders', '6 cylinders', '4 cylinders', '5 cylinders',
       'other', '3 cylinders', '10 cylinders', '12 cylinders'],
      dtype=object)

variables categoricas:

nominales: "manufacturer", "model", "fuel", "title_status", "transmission", "drive", "type", "paint_color", "state".

Ya que no tienen un orden lógico.

ordinales: "condition" y "cylinders".

condition: representa el estado del vehículo, ademas tiene una progresión de calidad que sigue una escala:
"salvage" < "fair" < "good" < "excellent" < "like new" < "new".
Estas categorías implican una mejora progresiva en la condición del vehículo, y por lo tanto, el orden importa.

cylinders: Se refiere a la cantidad de cilindros del motor. También tiene un orden natural:
"3 cylinders" < "4 cylinders" < "5 cylinders" < "6 cylinders" < "8 cylinders" < "10 cylinders" < "12 cylinders" < "other".
A mayor cantidad de cilindros, podria ser más potente es el motor y más caro el auto.

varibales numericas:

numericas: "year" y "odometer".
Ambas son númericas continuas, el año es un valor temporal medible, mientras que odometer, es un número que representa cuántos kilometros o millas ha recorrido el auto.

In [35]:
# Definir features y target
X = df_limpio.drop(columns="price")
y = df_limpio["price"]

In [36]:
# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Debido al gran tamaño del dataset original, el entrenamiento completo de Random Forest no terminana de ejecutar, intentado con estimadores de 100, luego 50 y por ultimo 20; por ende, se decidio buscar una alternativa para que se realizara la ejecucion de ambos modelos y poder analizarlos y elegir el mejor en base a sus metricas.

Para agilizar la experimentación, se optó por trabajar con una muestra aleatoria del 30% del conjunto de entrenamiento (frac=0.3), lo cual permite:

Reducir significativamente los tiempos de entrenamiento y evaluación, especialmente para modelos como RandomForestRegressor, que implican múltiples árboles y validaciones cruzadas.



In [37]:
# Reducción del set para acelerar entrenamientos
X_train_reducido = X_train.sample(frac=0.3, random_state=42)
y_train_reducido = y_train.loc[X_train_reducido.index]

In [38]:
# Columnas numéricas, ordinales y nominales
num_cols = ["year", "odometer"]
ord_cols = ["condition", "cylinders"]
nom_cols = ["manufacturer", "model", "fuel", "title_status", "transmission", "drive", "type", "paint_color", "state"]

In [39]:
# Pipeline para columnas numéricas: solo escalar.
num_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler())
])

# Pipeline para columnas ordinales: codificar y escalar.
ord_pipeline = Pipeline(steps=[
    ("ordinal", OrdinalEncoder(categories=[
        ["salvage", "fair", "good", "excellent", "like new", "new"],  # condition
        ["3 cylinders", "4 cylinders", "5 cylinders", "6 cylinders",
         "8 cylinders", "10 cylinders", "12 cylinders", "other"]     # cylinders
    ])),
    ("scaler", StandardScaler())
])

# Pipeline para columnas nominales: one-hot encoding
nom_pipeline = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

Modelo 1: Regresión Lineal (LinearRegression)

In [40]:
# ColumnTransformer: combina todos los preprocesamientos
preprocessor_lr = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_cols),
    ("ord", ord_pipeline, ord_cols),
    ("nom", nom_pipeline, nom_cols)
])

In [41]:
# Pipeline para Regresión Lineal
pipeline_lr = Pipeline(steps=[
    ("preprocessing", preprocessor_lr),
    ("regressor", LinearRegression())
])

In [43]:
# Entrenar el modelo
pipeline_lr.fit(X_train_reducido, y_train_reducido)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'odometer']),
                                                 ('ord',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder(categories=[['salvage',
                                                                                               'fair',
                                                                                               'good',
                                                                                               'excellent',
                                                                                               'like '
                                                                                               'new',
                                                                                               'new'],
                                                                                              ['3 '
                                                                                               'cylinders',
                                                                                               '4 '
                                                                                               'cylinders',
                                                                                               '5 '
                                                                                               'cylinders',
                                                                                               '6 '
                                                                                               'cylinders',
                                                                                               '8 '
                                                                                               'cylinders',
                                                                                               '10 '
                                                                                               'cylinders',
                                                                                               '12 '
                                                                                               'cylinders',
                                                                                               'other']])),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['condition', 'cylinders']),
                                                 ('nom',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['manufacturer', 'model',
                                                   'fuel', 'title_status',
                                                   'transmission', 'drive',
                                                   'type', 'paint_color',
                                                   'state'])])),
                ('regressor', LinearRegression())])

In [44]:
# Predecir
y_pred_lr = pipeline_lr.predict(X_test)

In [45]:
# Evaluar
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = mse_lr ** 0.5  # raíz cuadrada del MSE
r2_lr = r2_score(y_test, y_pred_lr)

Modelo 2: Bosque Aleatorio (RandomForestRegressor)

In [46]:
# ColumnTransformer: combina todos los preprocesamientos
preprocessor_rf = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_cols),
    ("ord", ord_pipeline, ord_cols),
    ("nom", nom_pipeline, nom_cols)
])

In [47]:
# Pipeline para Random Forest
pipeline_rf = Pipeline(steps=[
    ("preprocessing", preprocessor_rf),
    ("model", RandomForestRegressor(n_estimators=20, random_state=42))
])

In [48]:
# Entrenar el modelo
pipeline_rf.fit(X_train_reducido, y_train_reducido)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'odometer']),
                                                 ('ord',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder(categories=[['salvage',
                                                                                               'fair',
                                                                                               'good',
                                                                                               'excellent',
                                                                                               'like '
                                                                                               'new',
                                                                                               'new'],
                                                                                              ['3 '
                                                                                               'cylinders',
                                                                                               '4 '
                                                                                               'cylinders',
                                                                                               '5 '
                                                                                               'cylinders',
                                                                                               '6 '
                                                                                               'cylinders',
                                                                                               '8 '
                                                                                               'cylinders',
                                                                                               '10 '
                                                                                               'cylinders',
                                                                                               '12 '
                                                                                               'cylinders',
                                                                                               'other']])),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['condition', 'cylinders']),
                                                 ('nom',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['manufacturer', 'model',
                                                   'fuel', 'title_status',
                                                   'transmission', 'drive',
                                                   'type', 'paint_color',
                                                   'state'])])),
                ('model',
                 RandomForestRegressor(n_estimators=20, random_state=42))])

In [49]:
# Predecir
y_pred_rf = pipeline_rf.predict(X_test)

In [50]:
# Evaluar
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = mse_rf ** 0.5  # raíz cuadrada del MSE
r2_rf = r2_score(y_test, y_pred_rf)

In [51]:
# Mostrar resultados
print("Regresión Lineal")
print(f"  MSE:  {mse_lr:.2f}")
print(f"  RMSE: {rmse_lr:.2f}")
print(f"  R²:   {r2_lr:.4f}")

print("Random Forest Regressor")
print(f"  MSE:  {mse_rf:.2f}")
print(f"  RMSE: {rmse_rf:.2f}")
print(f"  R²:   {r2_rf:.4f}")

Regresión Lineal
  MSE:  81015824.52
  RMSE: 9000.88
  R²:   0.5740
Random Forest Regressor
  MSE:  51332097.92
  RMSE: 7164.64
  R²:   0.7301


Tras comparar los resultados de ambos modelos utilizando las métricas MSE, RMSE y R², se puede concluir que el Random Forest Regressor es el modelo con mejor desempeño, con valores de:
MSE:  51332097.92
RMSE: 7164.64
R²:   0.7301

El Random Forest logra un menor MSE y un mejor R².

Esto indica que explica mejor la variabilidad del precio y tiene predicciones más precisas en comparación con la regresión lineal.

**5. OPTIMIZACION DEL MODELO**

Como se comento anteriomente, la ejecución del random forest es más extensa, por lo cual toma más tiempo, por ende, se decide utilizar estimadores de 5 y 10 y un cv = 2 para tener menos validaciones cruzadas.

In [52]:
# Optimizacion de hiperparametros
forest_params = {
    "model__n_estimators": [5, 10]
}

forest_grid = GridSearchCV(pipeline_rf, forest_params, cv=2, scoring="r2")  # cv=2 para tener menos validaciones cruzadas
forest_grid.fit(X_train_reducido, y_train_reducido)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['year',
                                                                          'odometer']),
                                                                        ('ord',
                                                                         Pipeline(steps=[('ordinal',
                                                                                          OrdinalEncoder(categories=[['salvage',
                                                                                                                      'fair',
                                                                                                                      'good',
                                                                                                                      'excellent',
                                                                                                                      'like '
                                                                                                                      'new',
                                                                                                                      'new'],
                                                                                                                     ['3 '
                                                                                                                      'cylinders',
                                                                                                                      '4 '
                                                                                                                      'cylinders',
                                                                                                                      '5 '
                                                                                                                      'cylinders',
                                                                                                                      '6 '
                                                                                                                      'cylinders',
                                                                                                                      '8 '
                                                                                                                      'c...
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['condition',
                                                                          'cylinders']),
                                                                        ('nom',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['manufacturer',
                                                                          'model',
                                                                          'fuel',
                                                                          'title_status',
                                                                          'transmissi

In [53]:
# Evaluación.
forest_best = forest_grid.best_estimator_
y_pred_forest = forest_best.predict(X_test)

print("Random Forest Regressor")
print("Mejores parámetros:", forest_grid.best_params_)
print("R²:", r2_score(y_test, y_pred_forest))

Random Forest Regressor
Mejores parámetros: {'model__n_estimators': 10}
R²: 0.7201202515840618


La búsqueda encontró que el modelo con 10 árboles (n_estimators = 10) ofrecía el mejor rendimiento dentro de los valores evaluados.

Un R² de 0.7201 indica que el modelo optimizado es capaz de explicar aproximadamente el 72% de la variabilidad en los precios de los vehículos usados.

Aunque se consideró un rango reducido de estimadores (5 y 10), esta decisión fue adecuada para reducir considerablemente los tiempos de ejecución.

Si se contara con mayor capacidad de procesamiento o tiempo, se podría explorar un rango más amplio (como [50, 100, 200]) para buscar mejoras adicionales.

**CONCLUSIONES FINALES**

Se implementaron dos modelos de regresión para predecir el precio de vehículos usados:

Regresión Lineal:
R²: 0.5740
RMSE: 9000.88

Random Forest Regressor (sin optimizar):
R²: 0.7301
RMSE: 7164.64

Random Forest Regressor (optimizado con GridSearchCV):
Mejores parámetros:
n_estimators = 10
R²: 0.7201

El modelo Random Forest sin optimizar logró el mejor desempeño general con un R² de 0.7301, superando tanto a la regresión lineal como a la versión optimizada.

La versión optimizada con n_estimators = 10 tuvo un rendimiento ligeramente menor, con un R² de 0.7201, posiblemente debido a la baja cantidad de árboles utilizada para reducir el tiempo de ejecución.

Aun así, ambas versiones de Random Forest superan ampliamente a la regresión lineal, lo que confirma que un modelo no lineal se ajusta mejor a la complejidad del problema.

El mejor modelo para este problema es Random Forest Regressor sin optimizar, ya que entrega la mejor métrica R² y un menor RMSE.
La regresión lineal, aunque más simple y rápida, no logra modelar adecuadamente la variabilidad del precio de los vehículos.


De los gráficos se interpreta que:

La variable fuel puede ser una característica relevante para predecir el precio, ya que ciertos tipos de combustible tienden a asociarse con vehículos más caros o más nuevos.

La varibale price presenta una distribución no normal y sesgada, lo que justifica el uso de modelos más robustos como Random Forest. Además, es clave haber eliminado los outliers para evitar que afecten la escala y el aprendizaje del modelo.

La relación entre fuel, transmission y price sugiere que estas variables podrían ser importantes en los modelos de regresión. Además, la interacción entre variables categóricas podría ser aprovechada por modelos no lineales como Random Forest.
